In [1]:
from OLS.OLS import * 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D

%load_ext autoreload
%autoreload 2

plt.style.use('ggplot')

## Import data for EDA

Import Boston dataset with the next attributes columns:

1. *CRIM*      per capita crime rate by town
2. *ZN*        proportion of residential land zoned for lots over  25,000 sq.ft.
3. *INDUS*     proportion of non-retail business acres per town
4. *CHAS*      Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
5. *NOX*       nitric oxides concentration (parts per 10 million)
6. *RM*        average number of rooms per dwelling
7. *AGE*       proportion of owner-occupied units built prior to 1940
8. *DIS*       weighted distances to five Boston employment centres
9. *RAD*       index of accessibility to radial highways
10. *TAX*      full-value property-tax rate per \$10000
11. *PTRATIO*  pupil-teacher ratio by town
12. *BRATIO*        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
13. *LSTAT*    % lower status of the population
14. *MEDV*     Median value of owner-occupied homes in \$1000's


In [ ]:
dataset_name = 'housing.data'
dataset_columns = ['CR', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'BRATIO', 'LSTAT', 'MEDV']

dataset = pd.read_csv('datasets/housing.data', 
                      delim_whitespace=True, 
                      header=None, 
                      names=dataset_columns)

### Dataset analysis

Perform simple dataset analysis on correlation, atd.

In [ ]:
dataset.shape

In [ ]:
dataset.head()

In [ ]:
dataset.tail()

In [ ]:
dataset.describe()

In [ ]:
def colorCorrelatedFeatures(value):
    """
    Color higly correlated pairs of features
    """
    color = 'green' if abs(value) > 0.74 and abs(value) != 1 else 'white'
    return 'background-color: %s' % color
    

dataset.corr().style.applymap(colorCorrelatedFeatures)

### Let's investigate mostly correlated features

I will plot them as scatter plots to see the next points:

1. Mostly correlated parts of the graph
2. Possible outliers of the 4 plots
3. The necessity of scaling some axes

In [ ]:
def plot(axes, x_feature, y_feature, x_label, y_label, color, description):
    """
    Plots scatter plot with the specified parameters
    """
    axes.scatter(x_feature, y_feature, c=color)
    axes.set_title(description)
    axes.set_xlabel(x_label)
    axes.set_ylabel(y_label)

def plotFeatures():
    
    figure, (axes1, axes2) = plt.subplots(2, 2, figsize = (12, 12))

    description = "Nitric oxides concentration \nto proportion of non-retail business acres per town"
    plot(axes1[0], dataset['NOX'], dataset['INDUS'], 'NOX', 'INDUS', 'g', description)

    description = "Weighted distances to five Boston employment centres\n to nitric oxides concentration"
    plot(axes2[0], dataset['DIS'], dataset['NOX'], 'DIS', 'NOX', 'y', description)

    description = "Full-value property-tax rate per $10000 \nto index of accessibility to radial highways"
    plot(axes1[1], dataset['TAX'], dataset['RAD'], 'TAX', 'RAD', 'b', description)
    
    description = "Weighted distances to five Boston employment centrer \nto  proportion of owner-occupied units built prior to 1940"
    plot(axes2[1], dataset['DIS'], dataset['AGE'], 'DIS', 'AGE', 'r', description)
    
    figure.tight_layout()
    
    plt.show()

plotFeatures()

In [ ]:
def normalize(series: pd.Series) -> pd.Series:
    """
    Basic min-max normalization
    """
    return(series - series.min()) / (series.max() - series.min())

def filter_max(series: pd.Series, maximum_value):
    """
    Replaces outliers with the mean value
    """
    mean = series.mean()
    return series.apply(lambda x: x if x < maximum_value else mean)
    

Here we see that the mostly data is not clean and need some preparation:

### Plot NOX to INDUS:
As I see NOX is the percentage in range from 0 to 1, but mostly features as in range from 0 to 0.7, so remove outliers and normalise INDUS column

In [ ]:
dataset['NOX'] = filter_max(dataset['NOX'], 0.7)
dataset['INDUS'] = filter_max(dataset['INDUS'], 15)
dataset['INDUS'] = normalize(dataset['INDUS'])

### Plot NOX to the DIS
Here we can try to normalize dis feature

In [ ]:
dataset['DIS'] = normalize(dataset['DIS'])

### Plot RAD to TAX
Perform only basic min-max normalization, as removing outliers has shown a decrease in the high correlation between two features

In [ ]:
dataset['TAX'] = normalize(dataset['TAX'])
dataset['RAD'] = normalize(dataset['RAD'])

### Plot AGE to DIS
1. Age is the proportion from 0 to 100, so normalize to range 0 to 1
2. Dis also can be normalised to the range 0 to 1

In [ ]:
dataset['DIS'] = filter_max(dataset['DIS'], 0.8)
dataset['AGE'] = normalize(dataset['AGE'])

## Let's see the results 

In [ ]:
plotFeatures()

In [ ]:
dataset.corr().style.applymap(colorCorrelatedFeatures)

In [ ]:
dataset.head()

### OLS

Let's try to perform basic linear regression on the current features. 
OLS module supports different configurations and different regressions:

1. Basic linear regression, which is stastically calculated, using matrix operations
2. Gradient descent and its specifications, like (SGD and minibatch GD)

In [ ]:
train_data = dataset.sample(frac = 0.8)
test_data = dataset.drop(train_data.index)

### Linear regression predictions

Perform basic statistical linear regression:

This type of the regression provides the minimal MSE error and provides best fit for the data, however, the computation complexity is near O(n^2) to O(n^3) depending on the implementation of the matrix multiplication

In [ ]:
def create_linear_model(train_data, labels, target) -> OLS:
    """
    Create linear regression model and perform fit
    
    Input:
        labels - the list of the labels from the dataset
        targets - the name of the target from the dataset
    
    """
    model = OLS()
    model.fit(
        np.array(train_data[labels]), 
        np.array(train_data[[target]])
    )
    return model

def render_plot(axes, dataset: pd.DataFrame, prediction, 
                label: str, target: str, split: str):
    """
        Renders plot on the matplotlib.axes
        
        Input:
            axes - axes to draw on
            dataset - train and test datasets
            prediction - vector of predictions
            label - feature label
            target - target label
            split - string, which shows, which split is it
    """
    
    color = (np.random.rand(), np.random.rand(), np.random.rand())
    
    axes.scatter(dataset[[label]], dataset[[target]], c=[color])
    
    axes.set_title("Regression result between {} and {} ({})".format(label, target, split))
    axes.set_xlabel(label)
    axes.set_ylabel(target)
    
    axes.plot(
        dataset[[label]], 
        prediction,
        c = 'r'
    )

def visualize_linear_reg_result(model: OLS, train_data, test_data, label, target):
    """
    Draw a scatter plot with the prediction line:
    
    Input:
        label - the name of the label from the dataset
        target - the name of the target from the dataset
    """
    
    slopes = model.slopes
    
    prediction = model.predict(np.array(train_data[[label]]))
    print("Train data score: ", model.score(np.array(train_data[[target]]), prediction))
    
    figure, (axes1, axes2) = plt.subplots(1, 2, figsize = (12, 6))
    
    render_plot(axes1, train_data, prediction, label, target, 'train_data')

    prediction = model.predict(np.array(test_data[[label]]))
    print("Test data score: ", model.score(np.array(test_data[[target]]), prediction))
    
    render_plot(axes2, test_data, prediction, label, target, 'test_data')
    
    figure.tight_layout()
    
    plt.show()

In [ ]:
%%time
linear_model = create_linear_model(train_data, ['NOX'], 'INDUS')

In [ ]:
visualize_linear_reg_result(linear_model, train_data, test_data, 'NOX', 'INDUS')

In [ ]:
%%time
linear_model = create_linear_model(train_data, ['DIS'], 'NOX')

In [ ]:
visualize_linear_reg_result(linear_model, train_data, test_data, 'DIS', 'NOX')

In [ ]:
%%time
linear_model = create_linear_model(train_data, ['RAD'], 'TAX')

In [ ]:
visualize_linear_reg_result(linear_model, train_data, test_data, 'RAD', 'TAX')

In [ ]:
%%time
linear_model = create_linear_model(train_data, ['DIS'], 'AGE')

In [ ]:
visualize_linear_reg_result(linear_model, train_data, test_data, 'DIS', 'AGE')

# Gradient descent

Gradient descent is the approach, which is used to minimize some function by repeated iterated move in the direction of the minimum.

There are 3 basic parameters for GD:

1. Number of the iterations is the first one and it shows, how much iterations should be performed
2. Tolerance is the threshold of the difference in the cost function values. As we use GD, the cost function is the MSE.
3. Learning step tells the model, how fast should it move to the minimum of the function. The better the learning learning step is chosen, the faster the model will find the minimum. In comparison, a big learning step can result into overflow and a small learning step can significantly decrease the speed of the approximation.

Current implementation supports all these options and sets `slopes_records` and `cost_records` to visualize the process

Also OLS module supports:

1. Stochastic Gradient Descent is the gd, but takes only one sample from the training dataset per iteration. (It is the simplest implementation, where the learning step plays the key-role, other implementations can be found [here](https://en.wikipedia.org/wiki/Stochastic_gradient_descent)
2. Minibatch Gradient Descent is the sgd, but takes m samples from the trainit dataset per iteration.

In [ ]:
def create_gd_model(train_data, 
                    labels, 
                    target, 
                    number_of_iterations = 1000,
                    tolerance = 0.00001, 
                    learning_step = 0.001,
                    batch_size = 1,
                    type = 'GD') -> OLS:
    """
    Create gd model and perform fit
    
    Input:
        labels - the list of the labels from the dataset
        targets - the name of the target from the dataset
    
    """
    model = OLS(number_of_iterations, tolerance, learning_step, batch_size, type)
    model.fit(
        np.array(train_data[labels]), 
        np.array(train_data[[target]])
    )
    return model

In [ ]:
def render_approximation_process(model: OLS, train_data, label, target, skipOffset = 20):
    """
    Renders the process approximation using the cost records from the model
    
    Input:
        model - OLS model
        train_data - dataset for train
        label - label from the train dataset
        target - target label from the train dataset
        skipOffset - shows the number of records to skip
    """
    mse_errors = model.cost_records
    slopes_records = [model.slopes_records[0]]
    slopes_records += model.slopes_records[::skipOffset]
    slopes_records += [model.slopes_records[-1]]
    
    number_of_iterations = len(mse_errors)
    
    figure, (axes1, axes2) = plt.subplots(1, 2, figsize = (12, 6))
    
    axes1.set_title("MSE error")
    axes1.plot(np.arange(number_of_iterations), mse_errors)
    
    minimum = train_data[[label]].min()
    maximum = train_data[[label]].max()
    
    lin_space = np.linspace(minimum, maximum, 100)
    
    axes2.set_title("Learning process")
    axes2.scatter(train_data[[label]], train_data[target], c='r')
    
    for slope in enumerate(slopes_records):
        color = 'r' if slope[0] < len(slopes_records) - 1 else 'b'
        linewidth = 0.5 if slope[0] < len(slopes_records) - 1 else 1
        slopes = np.flip(slope[1].reshape(len(slope[1])), 0)
        function = np.poly1d(slopes)
        
        axes2.plot(lin_space, function(lin_space), linewidth = linewidth, c=color)

    figure.tight_layout()

### Video animation

Visualize the learning process at the speed of the 30 FPS and saves the output video to the file `videos/label_target_model_type_animation.mp4`. Add manually to the notebook, as execution can take time to execute.

In [ ]:
def animate_approximation_process(model: OLS, train_data, label, target, skipOffset = 1):
    """
    Renders the process approximation using the cost records from the model and
    saves the result to the basic_animation.mp4
    
    Input:
        model - OLS model
        train_data - dataset for train
        label - label from the train dataset
        target - target label from the train dataset
        skipOffset - shows the number of records to skip
    """
    mse_errors = model.cost_records
    slopes_records = [model.slopes_records[0]]
    slopes_records += model.slopes_records[::skipOffset]
    slopes_records += [model.slopes_records[-1]]
    
    number_of_iterations = len(slopes_records)
    
    figure, (axes1, axes2) = plt.subplots(1, 2, figsize = (12, 6))
    
    axes1.set_title("MSE error")
    axes1.set_xlabel("Number of iterations")
    axes1.set_ylabel("MSE error")
    
    mse_plot = axes1.plot()
    
    minimum = train_data[[label]].min()
    maximum = train_data[[label]].max()
    
    lin_space = np.linspace(minimum, maximum, 100)
    
    axes2.set_title("Learning process")
    axes2.scatter(train_data[[label]], train_data[target], c='r')
    axes2.set_xlabel(label)
    axes2.set_ylabel(target)
    axes2.set_ylim(bottom=train_data[target].min() - 0.1)
    
    line_plot, = axes2.plot(lin_space, lin_space)
    
    def animate(frame):
        # Render mse error
        if frame < len(mse_errors):
            axes1.plot(np.arange(number_of_iterations)[:frame], mse_errors[:frame], c='r')

        # Render slopes
        slope = slopes_records[frame]
        slopes = np.flip(slope.reshape(len(slope)), 0)
        function = np.poly1d(slopes)
        
        color = 'r' if frame < len(slopes_records) - 1 else 'y'
        linewidth = 0.5 if frame < len(slopes_records) - 1 else 1
        
        predictions = function(lin_space)

        line_plot.set_ydata(predictions)
        line_plot.set_color(color)
        line_plot.set_linewidth(linewidth)
        line_plot.set_alpha(0.8)
    
    animation = FuncAnimation(figure, animate, frames = number_of_iterations)
    animation.save('videos/{}-{}-{}.mp4'.format(label, target, model.type), fps=30, extra_args=['-vcodec', 'libx264'])
    return None

## NOX/INDUS

In [ ]:
%%time
gd_model = create_gd_model(train_data, ['NOX'], 'INDUS')

In [ ]:
visualize_linear_reg_result(gd_model, train_data, test_data, 'NOX', 'INDUS')
render_approximation_process(gd_model, train_data, 'NOX', 'INDUS')

In [ ]:
%%time
gd_model = create_gd_model(
    train_data, 
    ['NOX'], 
    'INDUS', 
    number_of_iterations=3000,
    learning_step=0.01, 
    type='SGD'
)

In [ ]:
visualize_linear_reg_result(gd_model, train_data, test_data, 'NOX', 'INDUS')
render_approximation_process(gd_model, train_data, 'NOX', 'INDUS', skipOffset = 100)

In [ ]:
%%time
gd_model = create_gd_model(
    train_data, 
    ['NOX'], 
    'INDUS', 
    batch_size=20,
    learning_step = 0.01,
    type='MGD'
)

In [ ]:
visualize_linear_reg_result(gd_model, train_data, test_data, 'NOX', 'INDUS')
render_approximation_process(gd_model, train_data, 'NOX', 'INDUS', skipOffset = 20)

## DIS/NOX

In [ ]:
%%time
gd_model = create_gd_model(train_data, ['DIS'], 'NOX')

In [ ]:
visualize_linear_reg_result(gd_model, train_data, test_data, 'DIS', 'NOX')
render_approximation_process(gd_model, train_data, 'DIS', 'NOX', skipOffset = 5)

In [ ]:
%%time
gd_model = create_gd_model(
    train_data, 
    ['DIS'], 
    'NOX',
    number_of_iterations=1000,
    tolerance = 0,
    learning_step=0.1, 
    type='SGD'
)

In [ ]:
visualize_linear_reg_result(gd_model, train_data, test_data, 'DIS', 'NOX')
render_approximation_process(gd_model, train_data, 'DIS', 'NOX', skipOffset = 100)

In [ ]:
%%time
gd_model = create_gd_model(
    train_data,
    ['DIS'], 
    'NOX', 
    tolerance=0,
    batch_size=40, 
    learning_step=0.01,
    type='MGD'
)

In [ ]:
visualize_linear_reg_result(gd_model, train_data, test_data, 'DIS', 'NOX')
render_approximation_process(gd_model, train_data, 'DIS', 'NOX', skipOffset = 20)

## RAD/TAX

In [ ]:
%%time
gd_model = create_gd_model(train_data, ['RAD'], 'TAX')

In [ ]:
visualize_linear_reg_result(gd_model, train_data, test_data, 'RAD', 'TAX')
render_approximation_process(gd_model, train_data, 'RAD', 'TAX', skipOffset = 5)

In [ ]:
%%time
gd_model = create_gd_model(
    train_data, 
    ['RAD'],
    'TAX',
    number_of_iterations=1000,
    learning_step=0.1,
    type='SGD'
)

In [ ]:
visualize_linear_reg_result(gd_model, train_data, test_data, 'RAD', 'TAX')
render_approximation_process(gd_model, train_data, 'RAD', 'TAX', skipOffset = 100)

In [ ]:
%%time
gd_model = create_gd_model(
    train_data,
    ['RAD'], 
    'TAX', 
    batch_size=40, 
    learning_step=0.01,
    type='MGD'
)

In [ ]:
visualize_linear_reg_result(gd_model, train_data, test_data, 'RAD', 'TAX')
render_approximation_process(gd_model, train_data, 'RAD', 'TAX', skipOffset = 20)

## DIS/AGE

In [ ]:
%%time
gd_model = create_gd_model(train_data, ['DIS'], 'AGE')

In [ ]:
visualize_linear_reg_result(gd_model, train_data, test_data, 'DIS', 'AGE')
render_approximation_process(gd_model, train_data, 'DIS', 'AGE', skipOffset = 5)

In [ ]:
%%time
gd_model = create_gd_model(
    train_data, 
    ['DIS'],
    'AGE',
    learning_step=0.1,
    type='SGD'
)

In [ ]:
visualize_linear_reg_result(gd_model, train_data, test_data, 'DIS', 'AGE')
render_approximation_process(gd_model, train_data, 'DIS', 'AGE', skipOffset = 100)

In [ ]:
%%time
gd_model = create_gd_model(
    train_data,
    ['DIS'], 
    'AGE', 
    batch_size=40, 
    learning_step=0.01,
    type='MGD'
)

In [ ]:
visualize_linear_reg_result(gd_model, train_data, test_data, 'DIS', 'AGE')
render_approximation_process(gd_model, train_data, 'DIS', 'AGE', skipOffset = 20)

### Conclusion

1. According to the results SGD and MGD, which are generally slower than the GD and simple linear regression. It is true, because the dataset is relatively small and on the bigger dataset with the bigger amount of the features, the result of SGD and MGD should win over GD and simple linear regression in time.
2. GD is more stable than MGD from the view of finding the minimum.

# At final, let's try to perform multiple-feature linear regression

We will predict the concentration of nitric oxides basing on the person age and it's distance to Boston centers.

### Distribution

In [ ]:
%matplotlib widget

def print_score(model, train_data, test_data, labels, target):
    prediction = model.predict(np.array(train_data[labels]))
    print("Train data score: ", model.score(np.array(train_data[[target]]), prediction))
    prediction = model.predict(np.array(test_data[labels]))
    print("Test data score: ", model.score(np.array(test_data[[target]]), prediction))

def render_3d(data, x_label, y_label, z_label):
    figure = plt.figure(figsize=(6, 6))
    axes = figure.add_subplot(111, projection='3d')
    axes.set_xlabel(x_label)
    axes.set_ylabel(y_label)
    axes.set_zlabel(z_label)
    axes.scatter(data[[x_label]], data[[y_label]], data[[z_label]], c='b', alpha=1, marker='^')

    figure.tight_layout()
    
    figure.show()
    
    return figure, axes
    
def render_3d_lin_reg_result(model, data, x_label, y_label, z_label):
    figure, axes = render_3d(data, x_label, y_label, z_label)
    print(np.array(model.predict(data[[x_label, y_label]])).flat)
    
    x = np.arange(0.0, 1.0, 0.02)
    y = np.arange(0.0, 1.0, 0.02)
    X, Y = np.meshgrid(x, y)
    
    Z = model.slopes[0] + model.slopes[1] * X + model.slopes[2] * Y
    
    axes.plot_surface(X, Y, Z, alpha=0.3)
    
    figure.tight_layout()
    
    figure.show()

In [ ]:
render_3d(train_data, 'DIS', 'AGE', 'NOX')

### Linear regression

In [ ]:
%%time
linear_model = create_linear_model(train_data, ['DIS', 'AGE'], 'NOX')

In [ ]:
print_score(linear_model, train_data, test_data, ['DIS', 'AGE'], 'NOX')

In [ ]:
render_3d_lin_reg_result(linear_model, train_data, 'DIS', 'AGE', 'NOX')

## GD

In [ ]:
%%time
gd_model = create_gd_model(
    train_data,
    ['DIS', 'AGE'], 
    'NOX',
    type='GD'
)

In [ ]:
print_score(gd_model, train_data, test_data, ['DIS', 'AGE'], 'NOX')

In [ ]:
render_3d_lin_reg_result(gd_model, train_data, 'DIS', 'AGE', 'NOX')

### SGD

In [ ]:
%%time
gd_model = create_gd_model(
    train_data, 
    ['DIS', 'AGE'],
    'NOX',
    number_of_iterations=3000,
    tolerance = 0.0000001,
    learning_step=0.01,
    type='SGD'
)

In [ ]:
print_score(gd_model, train_data, test_data, ['DIS', 'AGE'], 'NOX')

In [ ]:
render_3d_lin_reg_result(gd_model, train_data, 'DIS', 'AGE', 'NOX')

## MGD

In [ ]:
%%time
gd_model = create_gd_model(
    train_data,
    ['DIS', 'AGE'], 
    'NOX', 
    tolerance=0.000001,
    batch_size=40,
    learning_step=0.001,
    type='MGD'
)

In [ ]:
print_score(gd_model, train_data, test_data, ['DIS', 'AGE'], 'NOX')

In [ ]:
render_3d_lin_reg_result(gd_model, train_data, 'DIS', 'AGE', 'NOX')